<a href="https://colab.research.google.com/github/SireeshaM6/Advanced_House_Dataset/blob/main/Animal_Behaviour_Hypothesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agenda

1. Data Quality Check

In [ ]:
## Data Manipulation Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Preprocessing Libraries
from sklearn.preprocessing import StandardScaler


## Dataset validation Libraries
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold

In [ ]:
data = pd.read_csv('/content/abp_accel.csv')
data.head()

,timestamp,x,y,z
0,2015-06-12 13:30:00.161041,100,620,804
1,2015-06-12 13:30:00.260490,68,640,800
2,2015-06-12 13:30:00.359939,48,628,884
3,2015-06-12 13:30:00.459388,44,616,888
4,2015-06-12 13:30:00.558837,76,628,860


**Conclusions:**
Timestamp : yyyy-mm-dd hh:MM:ss.microseconds

Acceleration_Data : The accleration units are in Milligramgram force (MG)

In [ ]:
data.shape

(1441669, 4)

In [ ]:
14595853

## Duplicate Validation

In [ ]:
# Selecting duplicated rows except first occurence based on all columns
duplicate = data[data.duplicated()]
duplicate

,timestamp,x,y,z


**Conclusions:**

No duplicates are available

**NULL values validation**

In [ ]:
data.isnull().sum()

timestamp    0
x            0
y            0
z            1
dtype: int64

**Conclusions:**
Since we have only once null value record. We can remove that null value record.

In [ ]:
data = data.dropna()

In [ ]:
data.isnull().sum()

timestamp    0
x            0
y            0
z            0
dtype: int64

## Descriptive Statistics

In [ ]:
data.describe()

,x,y,z
count,1.329144e+07,1.329144e+07,1.329144e+07
mean,4.653656e+01,8.902614e+02,1.627622e+02
std,3.464005e+02,2.056476e+02,3.393956e+02
min,-1.840000e+03,-1.828000e+03,-2.040000e+03
25%,-2.400000e+02,8.280000e+02,-8.000000e+00
50%,1.680000e+02,9.680000e+02,1.200000e+02
75%,3.200000e+02,1.016000e+03,3.200000e+02
max,1.684000e+03,1.904000e+03,2.040000e+03


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13291436 entries, 0 to 13291435
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  object 
 1   x          int64  
 2   y          int64  
 3   z          float64
dtypes: float64(1), int64(2), object(1)
memory usage: 507.0+ MB


## Conclusions: Descriptive Statistics
From the descriptive Statistics, we can see mean and max values are having larger difference. It means we have outliers in the data.

## **Data Visualizations : Univariate Analysis**

In [ ]:
for col in data.columns:
  plt.figure(figsize=(8,4))
  ax = sns.histplot(data[col],kde=True,bins=20)
  plt.xticks(rotation=45)
  plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='x',data=data)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='y',data=data)
plt.show()

In [ ]:
plt.figure(figsize=(12,8))
sns.boxplot(x='z',data=data)
plt.show()

In [ ]:
data.shape

In [ ]:
data_df = data[data['z'] >= -1000]
data_df.shape

**Conclusions**

From the Z feature, felt we have outliers in the lower whisker. Removed those 4 rows

In [ ]:
sns.barplot(x='x',y='y',hue='z',data = data_df)
plt.show()

**Convert the Object to Datetime**

In [ ]:
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'],format='%Y-%m-%d %H:%M:%S.%f')


In [ ]:
data_df.head()

In [ ]:
data_df.head(13)

In [ ]:
data_df.info()

In [ ]:
# sns.pairplot(data_df)
# plt.show()

**Conclusions:**
Since the distribution of the data is very dense like above. We can use DBSCAN for the Clustering purpose.

In [ ]:
for col in data_df.columns:
  plt.figure(figsize=(8,4))
  ax = sns.histplot(data_df[col],kde=True,bins=20)
  plt.xticks(rotation=45)
  plt.show()

## Multicollinearity Check

In [ ]:
data_df1 = data_df.drop(['timestamp'],axis=1)

In [ ]:
vif_data = pd.DataFrame()
vif_data['feature'] = data_df1.columns


# Calculating the VIF for each feature
vif_data['VIF'] = [variance_inflation_factor(data_df1.values,i) for i in range(len(data_df1.columns))]

vif_data = vif_data.sort_values('VIF',ascending=False)

vif_data


In [ ]:
plt.figure(figsize=(10,7))
ax = sns.barplot(y='feature',x ='VIF',data = vif_data)
plt.show()

## Conclusions : Multicollinearity Validation

Since the VIF score is less than 10, All features are independent to each other

# Importance of each feature

In [ ]:
corr = data_df.corr()
top_features = corr.index
plt.figure(figsize=(6,6))
sns.heatmap(data_df[top_features].corr(),annot=True)
plt.show()

## Conclusions : Pearson Correlation Matrix

We could see a Strongly negatively correlated. Means if one feature increases another featuer decreases.

## Feature Selection - Dropping Constant Features.
It helps to find the low variance features.

In [ ]:
var_thres = VarianceThreshold(threshold = 0)
var_thres.fit(data_df1)

In [ ]:
var_thres.get_support()

In [ ]:
data_df1.columns[var_thres.get_support()]

In [ ]:
constant_columns = [column for column in data_df1.columns if column not in data_df1.columns[var_thres.get_support()]]


print(constant_columns)

## Conclusions : Constant Features
In the Dataset, we don't see any constant features. The variance of the features are very good.